In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
drive_PATH = '../content/drive/MyDrive/Colab Notebooks/l101.experiments.1'

Mounted at /content/drive


In [2]:
!pip install jsonlines scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 1.3 MB/s eta 0:00:00


In [3]:
import jsonlines

DATA_NAME = 'DEEN'
# DATA_NAME = 'ENDE'
# DATA_NAME = 'ENTR'
# DATA_NAME = 'TREN'
DATA_PATH = drive_PATH + f'/data-main/{DATA_NAME}.jsonl'
DATA = []
with jsonlines.open(DATA_PATH) as f:
    for line in f.iter():
        DATA.append(line)

In [4]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
import tqdm
import scipy.stats as ss
import matplotlib.pyplot as plt
import seaborn as sns
import random
import sys

In [5]:
num_datapoints = 1000
reduced_dim = 10

In [6]:
# Encoded Data
enc_candidates = []
enc_candidates10 = []
# Scores
BLEURT_cand_scores = []
CHRF_cand_scores = []
# chrF++ Scores
MBR_BLEURT_cand_scores = []
MBR_CHRF_cand_scores = []
# Winner Indexes
MBR_BLEURT_winner_index = []
MBR_CHRF_winner_index = []
# Blackbox Functions
enc10_to_idx = []
enc10_to_MBR_BLEURT = []
enc10_to_MBR_CHRF = []
# Rankings
MBR_BLEURT_index_rank = []
MBR_CHRF_index_rank = []

for idx in tqdm.tqdm(range(num_datapoints)):
    idx_datapoint = DATA[idx]

    # Encoded Data
    _enc_data = np.array(idx_datapoint['enc_candidates'])
    enc_candidates.append(_enc_data)
    pca = PCA()
    _enc_data_10 = pca.fit_transform(_enc_data)[:,:reduced_dim]
    enc_candidates10.append(_enc_data_10)

    # Scores
    _bleurt_scores = np.array([np.round(score, 6)-0.5 for score in idx_datapoint['BLEURT_cand_scores']])
    _chrf_scores = np.array([np.round(score, 6)-50 for score in idx_datapoint['CHRF_cand_scores']])
    BLEURT_cand_scores.append(_bleurt_scores)
    CHRF_cand_scores.append(_chrf_scores)

    # MBR Scores
    _mbr_bleurt_scores = np.array([np.round(score, 6) for score in idx_datapoint['MBR_BLEURT_cand_scores']])/128 -0.5
    _mbr_chrf_scores = np.array([np.round(score, 6) for score in idx_datapoint['MBR_CHRF_cand_scores']])/128 -50
    MBR_BLEURT_cand_scores.append(_mbr_bleurt_scores)
    MBR_CHRF_cand_scores.append(_mbr_chrf_scores)

    # Winner Indexes
    _mbr_bleurt_winner_idx = idx_datapoint['MBR_BLEURT_winner_index']
    _mbr_chrf_winner_idx = idx_datapoint['MBR_CHRF_winner_index']
    MBR_BLEURT_winner_index.append(_mbr_bleurt_winner_idx)
    MBR_CHRF_winner_index.append(_mbr_chrf_winner_idx)

    # Blackbox Function : Uses Negative Utility = Risk
    _enc10_to_idx = {str(_enc_data_10[i]): i for i in range(128)}
    _enc10_to_MBR_BLEURT = {str(enc10):_mbr_bleurt_scores[_enc10_to_idx[str(enc10)]] for enc10 in _enc10_to_idx.keys()}
    _enc10_to_MBR_chrF = {str(enc10):_mbr_chrf_scores[_enc10_to_idx[str(enc10)]] for enc10 in _enc10_to_idx.keys()}

    enc10_to_idx.append(_enc10_to_idx)
    enc10_to_MBR_BLEURT.append(_enc10_to_MBR_BLEURT)
    enc10_to_MBR_CHRF.append(_enc10_to_MBR_chrF)

    # Rankings
    _MBR_BLEURT_index_rank = ss.rankdata(_mbr_bleurt_scores)
    _MBR_CHRF_index_rank = ss.rankdata(_mbr_chrf_scores)
    MBR_BLEURT_index_rank.append(_MBR_BLEURT_index_rank)
    MBR_CHRF_index_rank.append(_MBR_CHRF_index_rank)

enc_candidates = np.array(enc_candidates)
enc_candidates10 = np.array(enc_candidates10)
BLEURT_cand_scores = np.array(BLEURT_cand_scores)
CHRF_cand_scores = np.array(CHRF_cand_scores)
MBR_BLEURT_cand_scores = np.array(MBR_BLEURT_cand_scores)
MBR_CHRF_cand_scores = np.array(MBR_CHRF_cand_scores)
MBR_BLEURT_winner_index = np.array(MBR_BLEURT_winner_index)
MBR_CHRF_winner_index = np.array(MBR_CHRF_winner_index)
### Blackbox Function Example Usage
# enc10_to_MBR_chrF[0][str(enc_candidates10[0][0])]

100%|██████████| 1000/1000 [01:43<00:00,  9.64it/s]


In [7]:
from skopt.learning import GaussianProcessRegressor
from skopt.space import Space
from skopt.acquisition import gaussian_ei
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel
import numpy as np

from scipy.stats import norm

np.random.seed(1)

In [8]:
BASELINE_best_MBR_CHRF = []
BASELINE_best_MBR_CHRF_rank = []
BASELINE_best_CHRF = []

MODEL_best_MBR_CHRF = []
MODEL_best_MBR_CHRF_rank = []
MODEL_best_CHRF = []

for experiment in range(10):
    for idx in tqdm.tqdm(range(1000)):
        # CHRF
        X = enc_candidates10[idx]
        y = MBR_CHRF_cand_scores[idx]
        f = enc10_to_MBR_CHRF[idx]

        #######################
        ### Random Baseline ###
        #######################
        n_samples = 10
        initial_indices = np.random.choice(range(len(X)), n_samples, replace=False)
        initial_samples = X[initial_indices]
        initial_evaluations = [f[str(x)] for x in initial_samples]
        best_index_local = np.argmax(initial_evaluations)
        best_index = initial_indices[best_index_local]
        # Results
        best_MBR_CHRF = MBR_CHRF_cand_scores[idx][best_index]
        best_MBR_CHRF_rank = MBR_CHRF_index_rank[idx][best_index]
        best_CHRF = CHRF_cand_scores[idx][best_index]
        # print("Baseline - best_MBR_CHRF:", best_MBR_CHRF)
        # print("Baseline - best_MBR_CHRF_rank:", best_MBR_CHRF_rank)
        # print("Baseline - best_CHRF:", best_CHRF)
        BASELINE_best_MBR_CHRF.append(best_MBR_CHRF)
        BASELINE_best_MBR_CHRF_rank.append(best_MBR_CHRF_rank)
        BASELINE_best_CHRF.append(best_CHRF)
        #####################################################################################
        ### Bayesian Optimization with Gaussian Process Regression - Expected Improvement ###
        #####################################################################################
        n_initial_points = 5
        num_iterations = 5
        initial_indices = np.random.choice(range(len(X)), n_initial_points, replace=False)
        initial_samples = X[initial_indices]
        initial_evaluations = [f[str(x)] for x in initial_samples]
        # GP model
        kernel = ConstantKernel(constant_value=1.0) * RBF(length_scale=1.0)
        gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5 , normalize_y =False,  alpha=0.00001)
        # Train GP Model
        gp.fit(initial_samples, initial_evaluations)

        for _ in range(num_iterations):
            # Current best
            current_best = np.max(gp.y_train_)
            # Acquisition values
            acquisition_values = - gaussian_ei(X, gp, y_opt=current_best, xi=0.01)
            # acquisition_values = np.zeros([128]) # Random Baseline
            # Filter out previously evaluated points
            evaluated_points = set(map(tuple, gp.X_train_))
            filtered_acquisition = np.array([acq if tuple(point) not in evaluated_points else -np.inf for point, acq in zip(X, acquisition_values)])
            ##########################
            # Select the next index (Alternative 1)
            # next_index = np.argmax(filtered_acquisition)
            # Select the next index (Alternative 2)
            p_current = n_initial_points + _ + 100   # Filter out smallest elements p elements
            smallest_ten_indices = np.argsort(filtered_acquisition)[:p_current]
            for index in smallest_ten_indices:
                filtered_acquisition[index] = float('-inf')
            valid_indices = [index for index, value in enumerate(filtered_acquisition) if value != float('-inf')] # indices whose corresponding elements are not -inf
            next_index = random.choice(valid_indices) #  sample an index from the valid indices
            ##########################
            # Select the next point
            next_point = X[next_index]
            # Evaluate the next point
            next_evaluation = f[str(next_point)]
            # Train GP model including the new point
            X_new = np.vstack([gp.X_train_, next_point])
            Y_new = np.append(gp.y_train_, next_evaluation)
            gp.fit(X_new,Y_new)

        # Results
        best_evaluation_index_local = np.argmax(gp.y_train_)
        best_point = gp.X_train_[best_evaluation_index_local]
        best_value = gp.y_train_[best_evaluation_index_local]
        best_evaluation_index = enc10_to_idx[idx][str(best_point)]
        best_MBR_CHRF = MBR_CHRF_cand_scores[idx][best_evaluation_index]
        best_MBR_CHRF_rank = MBR_CHRF_index_rank[idx][best_evaluation_index]
        best_CHRF = CHRF_cand_scores[idx][best_evaluation_index]
        # print("Model best_MBR_CHRF:", best_MBR_CHRF)
        # print("Model best_MBR_CHRF_rank:", best_MBR_CHRF_rank)
        # print("Model best_CHRF:", best_CHRF)
        MODEL_best_MBR_CHRF.append(best_MBR_CHRF)
        MODEL_best_MBR_CHRF_rank.append(best_MBR_CHRF_rank)
        MODEL_best_CHRF.append(best_CHRF)
        # print('---')

Streaming output truncated to the last 5000 lines.
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
 29%|██▊       | 287/1000 [03:16<07:10,  1.66it/s]/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
 29%|██▉       | 291/1000 [03:19<07:20,  1.61it/s]/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modu

In [9]:
print('BASELINE_best_MBR_CHRF:',np.mean(BASELINE_best_MBR_CHRF)+50)
print('BASELINE_best_MBR_CHRF_rank:',np.mean(BASELINE_best_MBR_CHRF_rank))
print('BASELINE_best_CHRF:',np.mean(BASELINE_best_CHRF)+50)

print('MODEL_best_MBR_CHRF:',np.mean(MODEL_best_MBR_CHRF)+50)
print('MODEL_best_MBR_CHRF_rank:',np.mean(MODEL_best_MBR_CHRF_rank))
print('MODEL_best_CHRF:',np.mean(MODEL_best_CHRF)+50)

print('UPPER_BOUND MBR CHRF:', MBR_CHRF_cand_scores[idx][MBR_CHRF_winner_index[idx]]+50)
print('UPPER_BOUND RANK:', MBR_CHRF_index_rank[idx][MBR_CHRF_winner_index[idx]])
print('UPPER_BOUND CHRF:', CHRF_cand_scores[idx][MBR_CHRF_winner_index[idx]]+50)

BASELINE_best_MBR_CHRF: 65.32908414203125
BASELINE_best_MBR_CHRF_rank: 116.80035
BASELINE_best_CHRF: 56.640187346
MODEL_best_MBR_CHRF: 65.7242615425086
MODEL_best_MBR_CHRF_rank: 118.9439
MODEL_best_CHRF: 57.0228221086
UPPER_BOUND MBR CHRF: 72.9105303671875
UPPER_BOUND RANK: 128.0
UPPER_BOUND CHRF: 65.065986


In [10]:
n = len(BASELINE_best_MBR_CHRF_rank)
print(n)
print('Baseline is Better', sum([BASELINE_best_MBR_CHRF_rank[i] > MODEL_best_MBR_CHRF_rank[i] for i in range(n)]))
print('Model is Better', sum([BASELINE_best_MBR_CHRF_rank[i] < MODEL_best_MBR_CHRF_rank[i] for i in range(n)]))
print('Tie', sum([BASELINE_best_MBR_CHRF_rank[i] == MODEL_best_MBR_CHRF_rank[i] for i in range(n)]))

10000
Baseline is Better 3518
Model is Better 5042
Tie 1440
